In [1]:

from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

c:\Users\HP\Desktop\mywork\NOTEBOOKS\working_with_llms\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")


In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=openai_api_key
)

In [4]:
# System prompt that defines assistant behavior
sys_msg = SystemMessage(
    content="You are a helpful customer support representative answering user questions. Be helpful and concise."
)

def assistant(state: MessagesState) -> dict:
    """
    The assistant node - processes messages and generates response.
    """
    # Combine system prompt with conversation history
    messages = [sys_msg] + state["messages"]
    
    # Get response from LLM
    response = llm.invoke(messages)
    
    # Return as state update
    return {"messages": [AIMessage(content=response.content)]}

In [5]:
# Create a StateGraph with MessagesState
builder = StateGraph(MessagesState)

# Add the assistant node
builder.add_node("assistant", assistant)

# Define the flow:
# START → assistant → END
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

# Create a memory checkpointer (stores in memory)
memory = MemorySaver()

# Compile the graph WITH memory
agent = builder.compile(checkpointer=memory)

In [6]:
# Define a session ID for this conversation
session_id = "chat-session-001"

def run_conversation(user_input: str, thread_id: str = session_id):
    """
    Send a message to the agent and get response.
    ⚠️ WARNING: Using default thread_id shares conversation acrosss all calls!
    In production, ALWAYS provide unique thread_id per user.
    """
    # Invoke the agent
    result = agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Print the conversation
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            print(f"\n👤 User: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 Agent: {message.content}")
    
    print("\n" + "="*70)

# First message
run_conversation("I bought a charger")

# Follow-up question - does it remember?
run_conversation("It's not working, what should I do?")


👤 User: I bought a charger
🤖 Agent: Great! If you have any questions about your charger, such as compatibility, usage, or troubleshooting, feel free to ask!


👤 User: I bought a charger
🤖 Agent: Great! If you have any questions about your charger, such as compatibility, usage, or troubleshooting, feel free to ask!

👤 User: It's not working, what should I do?
🤖 Agent: Here are some steps you can take to troubleshoot your charger:

1. **Check the Outlet:** Make sure the outlet you are using is working by testing it with another device.
2. **Inspect the Charger:** Look for any visible damage on the charger, such as frayed wires or bent connectors.
3. **Try a Different Device:** Plug the charger into a different device to see if it charges.
4. **Heat Check:** If the charger gets unusually hot, stop using it and consider replacing it.
5. **Reset the Device:** If applicable, try resetting the device you're charging.

If none of these steps work, you might want to contact the retailer or man